In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
%cd gdrive/My\ Drive/rubik_project

/content/gdrive/My Drive/rubik_project


In [3]:
!nvidia-smi

Thu Jan 23 12:54:30 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
%tensorflow_version 1.x
import tensorflow as tf
import data_utils
import data_provider 
import model
import config

In [0]:
sess = tf.InteractiveSession()

In [6]:
loaded_model = model.Multitask_BKNet(sess, False)




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

(?, 24, 24, 32)
(?, 12, 12, 64)
(?, 6, 6, 128)
(?, 3, 3, 256)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
keep_dims is deprecated, use keepdims instead



Instructions for updating:
Please use tf.global_variables instead.
Reading model parameters from %s /content/gdrive/My Drive/rubik_project/checkpoints/23012020_3branches_svm/model.ckpt
INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/rubik_project/checkpoints/23012020_3branches_svm/model.ckpt


In [7]:
public_test_data = data_provider.Dataset('public_test', config.BATCH_SIZE)

Load smile image...................
Done !
Number of smile train data:  3000
---------------------------------------------------------------
Load emotion image..................
Done !
Number of emotion train data:  28709
---------------------------------------------------------------
Load gender image...................
Done !
Number of gender train data:  30000
---------------------------------------------------------------


In [8]:
smile_nb_true_pred = 0
emotion_nb_true_pred = 0
gender_nb_true_pred = 0

smile_nb_test = 0
emotion_nb_test = 0
gender_nb_test = 0

for batch_image, batch_label, batch_index in public_test_data.gen():
  for i in range(len(batch_index)):
    if batch_index[i] == 1.0:
      smile_nb_test += 1
    elif batch_index[i] == 2.0:
      emotion_nb_test += 1
    else:
      gender_nb_test += 1
  
  feed_dict = {loaded_model.input_images: batch_image,
               loaded_model.input_labels: batch_label,
               loaded_model.input_indexes: batch_index,
               loaded_model.phase_train: True,
               loaded_model.keep_prob: 0.5}
  smile_nb_true_pred += loaded_model.sess.run(loaded_model.smile_true_pred, feed_dict=feed_dict)
  emotion_nb_true_pred += loaded_model.sess.run(loaded_model.emotion_true_pred, feed_dict=feed_dict)
  gender_nb_true_pred += loaded_model.sess.run(loaded_model.gender_true_pred, feed_dict=feed_dict)

smile_public_test_accuracy = smile_nb_true_pred * 1.0 / smile_nb_test
emotion_public_test_accuracy = emotion_nb_true_pred * 1.0 / emotion_nb_test
gender_public_test_accuracy = gender_nb_true_pred * 1.0 / gender_nb_test

print('\n')
print('Smile task public test accuracy: ' + str(smile_public_test_accuracy * 100))
print('Emotion task public test accuracy: ' + str(emotion_public_test_accuracy * 100))
print('Gender task public test accuracy: ' + str(gender_public_test_accuracy * 100))



Smile task public test accuracy: 92.0
Emotion task public test accuracy: 57.45332961827807
Gender task public test accuracy: 89.4851869839728


In [9]:
private_data = data_provider.Dataset('private_test', config.BATCH_SIZE)

Load smile image...................
Done !
Number of smile train data:  3000
---------------------------------------------------------------
Load emotion image..................
Done !
Number of emotion train data:  28709
---------------------------------------------------------------
Load gender image...................
Done !
Number of gender train data:  30000
---------------------------------------------------------------


In [10]:
smile_nb_true_pred = 0
emotion_nb_true_pred = 0
gender_nb_true_pred = 0

smile_nb_test = 0
emotion_nb_test = 0
gender_nb_test = 0

for batch_image, batch_label, batch_index in private_data.gen():
  for i in range(len(batch_index)):
    if batch_index[i] == 1.0:
      smile_nb_test += 1
    elif batch_index[i] == 2.0:
      emotion_nb_test += 1
    else:
      gender_nb_test += 1
  
  feed_dict = {loaded_model.input_images: batch_image,
               loaded_model.input_labels: batch_label,
               loaded_model.input_indexes: batch_index,
               loaded_model.phase_train: True,
               loaded_model.keep_prob: 0.5}
  smile_nb_true_pred += loaded_model.sess.run(loaded_model.smile_true_pred, feed_dict=feed_dict)
  emotion_nb_true_pred += loaded_model.sess.run(loaded_model.emotion_true_pred, feed_dict=feed_dict)
  gender_nb_true_pred += loaded_model.sess.run(loaded_model.gender_true_pred, feed_dict=feed_dict)

smile_private_test_accuracy = smile_nb_true_pred * 1.0 / smile_nb_test
emotion_private_test_accuracy = emotion_nb_true_pred * 1.0 / emotion_nb_test
gender_private_test_accuracy = gender_nb_true_pred * 1.0 / gender_nb_test

print('\n')
print('Smile task private test accuracy: ' + str(smile_private_test_accuracy * 100))
print('Emotion task private test accuracy: ' + str(emotion_private_test_accuracy * 100))
print('Gender task private test accuracy: ' + str(gender_private_test_accuracy * 100))



Smile task private test accuracy: 91.9
Emotion task private test accuracy: 58.93006408470326
Gender task private test accuracy: 89.46090335114133
